<a href="https://colab.research.google.com/github/MLutfor/Home_Sales/blob/main/Home_Sales_starter_code_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=11006c5f31a5179469e3dc93808bb4be3975be0d9e3d5d8316724a02764f962a
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [ ]:
# Import packages
from pyspark.sql import SparkSession

from pyspark.sql import functions as F
from pyspark.sql.types import StringType
import pandas as pd

import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [ ]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)
df.show()


+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [ ]:
# 2. Create a temporary view of the DataFrame.

df.createOrReplaceTempView("home_sales_view")

In [ ]:
# 3. What is the average price for a four-bedroom house sold in each year rounded to two decimal places?
query = """
    SELECT
        YEAR(date) AS year,
        AVG(price) AS avg_price
    FROM
        home_sales_view
    WHERE
        bedrooms = 4
    GROUP BY
        year
    ORDER BY
        year
"""

result = spark.sql(query)
result.show(truncate=False)



+----+------------------+
|year|avg_price         |
+----+------------------+
|2019|300263.6955128205 |
|2020|298353.78003169573|
|2021|301819.44398864   |
|2022|296363.8845050215 |
+----+------------------+



In [ ]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
query = """
    SELECT
        date_built AS year_built,
        AVG(price) AS avg_price
    FROM
        home_sales_view
    WHERE
        bedrooms = 3 AND bathrooms = 3
    GROUP BY
        year_built
    ORDER BY
        year_built
"""

result = spark.sql(query)
result.show(truncate=False)


+----------+------------------+
|year_built|avg_price         |
+----------+------------------+
|2010      |292859.615942029  |
|2011      |291117.46706586826|
|2012      |293683.1872074883 |
|2013      |295962.27145085804|
|2014      |290852.2661870504 |
|2015      |288770.2966101695 |
|2016      |290555.073964497  |
|2017      |292676.7887740029 |
+----------+------------------+



In [ ]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors, and are greater than or equal to 2,000 square feet rounded to two decimal places?
query = """
    SELECT
        date_built AS year_built,
        AVG(price) AS avg_price
    FROM
        home_sales_view
    WHERE
        bedrooms = 3 AND
        bathrooms = 3 AND
        floors = 2 AND
        sqft_living >= 2000
    GROUP BY
        year_built
    ORDER BY
        year_built
"""

result = spark.sql(query)
result.show(truncate=False)



+----------+------------------+
|year_built|avg_price         |
+----------+------------------+
|2010      |285010.2215909091 |
|2011      |276553.8128654971 |
|2012      |307539.97402597405|
|2013      |303676.79375      |
|2014      |298264.7183908046 |
|2015      |297609.9679144385 |
|2016      |293965.1046511628 |
|2017      |280317.57692307694|
+----------+------------------+



In [ ]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()



print("--- %s seconds ---" % (time.time() - start_time))

--- 7.200241088867188e-05 seconds ---


In [16]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than or equal to $350,000?
start_time = time.time()

# Run Spark SQL query
result = spark.sql("""
    SELECT
        view,
        ROUND(AVG(price), 2) AS avg_price
    FROM
        home_sales_view
    WHERE
        price >= 350000
    GROUP BY
        view
""")

result.show(truncate=False)

print("--- %s seconds ---" % (time.time() - start_time))

+----+----------+
|view|avg_price |
+----+----------+
|7   |403005.77 |
|51  |788128.21 |
|15  |404673.3  |
|54  |798684.82 |
|11  |399548.12 |
|29  |397771.65 |
|69  |750537.94 |
|42  |396964.5  |
|87  |1072285.2 |
|73  |752861.18 |
|64  |767036.67 |
|3   |398867.6  |
|30  |397862.0  |
|34  |401419.75 |
|59  |791453.0  |
|8   |398592.71 |
|28  |402124.62 |
|22  |402022.68 |
|85  |1056336.74|
|16  |399586.53 |
+----+----------+
only showing top 20 rows

--- 1.0174167156219482 seconds ---


In [17]:
# 7. Cache the the temporary table home_sales.
spark.sql("CACHE TABLE home_sales_view")

DataFrame[]

In [20]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales_view')

True

In [ ]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()



print("--- %s seconds ---" % (time.time() - start_time))


--- 5.459785461425781e-05 seconds ---


In [21]:
# 9. Using the cached data, run the query that filters out the view ratings with average price greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.
start_time_cached = time.time()

# Run Spark SQL query using cached data
result_cached = spark.sql("""
    SELECT
        view,
        ROUND(AVG(price), 2) AS avg_price
    FROM
        home_sales_view
    WHERE
        price >= 350000
    GROUP BY
        view
""")

result_cached.show(truncate=False)

print("--- Cached Runtime: %s seconds ---" % (time.time() - start_time_cached))

# Uncached Runtime (using the same query as before)
start_time_uncached = time.time()

result_uncached = df.filter(df["price"] >= 350000).groupBy("view").agg(F.round(F.avg("price"), 2).alias("avg_price"))
result_uncached.show(truncate=False)

print("--- Uncached Runtime: %s seconds ---" % (time.time() - start_time_uncached))


+----+----------+
|view|avg_price |
+----+----------+
|7   |403005.77 |
|51  |788128.21 |
|15  |404673.3  |
|54  |798684.82 |
|11  |399548.12 |
|29  |397771.65 |
|69  |750537.94 |
|42  |396964.5  |
|87  |1072285.2 |
|73  |752861.18 |
|64  |767036.67 |
|3   |398867.6  |
|30  |397862.0  |
|34  |401419.75 |
|59  |791453.0  |
|8   |398592.71 |
|28  |402124.62 |
|22  |402022.68 |
|85  |1056336.74|
|16  |399586.53 |
+----+----------+
only showing top 20 rows

--- Cached Runtime: 0.7688357830047607 seconds ---
+----+----------+
|view|avg_price |
+----+----------+
|7   |403005.77 |
|51  |788128.21 |
|15  |404673.3  |
|54  |798684.82 |
|11  |399548.12 |
|29  |397771.65 |
|69  |750537.94 |
|42  |396964.5  |
|87  |1072285.2 |
|73  |752861.18 |
|64  |767036.67 |
|3   |398867.6  |
|30  |397862.0  |
|34  |401419.75 |
|59  |791453.0  |
|8   |398592.71 |
|28  |402124.62 |
|22  |402022.68 |
|85  |1056336.74|
|16  |399586.53 |
+----+----------+
only showing top 20 rows

--- Uncached Runtime: 0.653516530

In [22]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data

df.write.partitionBy("date_built").mode("overwrite").parquet("home_sales_partitioned_by_date_built")

In [23]:
# 11. Read the parquet formatted data.
p_df_p=spark.read.parquet('home_sales_partitioned_by_date_built')

# Show the DataFrame
p_df_p.show()

+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|                  id|      date| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|date_built|
+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|2ed8d509-7372-46d...|2021-08-06|258710|       3|        3|       1918|    9666|     1|         0|  25|      2015|
|941bad30-eb49-4a7...|2020-05-09|229896|       3|        3|       2197|    8641|     1|         0|   3|      2015|
|c797ca12-52cd-4b1...|2019-06-08|288650|       2|        3|       2100|   10419|     2|         0|   7|      2015|
|0cfe57f3-28c2-472...|2019-10-04|308313|       3|        3|       1960|    9453|     2|         0|   2|      2015|
|d715f295-2fbf-4e9...|2021-05-17|391574|       3|        2|       1635|    8040|     2|         0|  10|      2015|
|a18515a2-86f3-46b...|2022-02-18|419543|       3|        2|       1642|   12826|

In [24]:
# 12. Create a temporary table for the parquet data.
p_df_p.createOrReplaceTempView("parquet_home_sales")



In [ ]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

start_time = time.time()



print("--- %s seconds ---" % (time.time() - start_time))

--- 7.104873657226562e-05 seconds ---


In [25]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000 with the Parquet DataFrame.
start_time_parquet = time.time()

# Run Spark SQL query using Parquet DataFrame
result_parquet = spark.sql("""
    SELECT
        view,
        ROUND(AVG(price), 2) AS avg_price
    FROM
        parquet_home_sales
    WHERE
        price >= 350000
    GROUP BY
        view
""")

result_parquet.show(truncate=False)

print("--- Parquet Runtime: %s seconds ---" % (time.time() - start_time_parquet))

# Cached Runtime (using the same query as before)
start_time_cached = time.time()

result_cached = spark.sql("""
    SELECT
        view,
        ROUND(AVG(price), 2) AS avg_price
    FROM
        home_sales_view
    WHERE
        price >= 350000
    GROUP BY
        view
""")

result_cached.show(truncate=False)

print("--- Cached Runtime: %s seconds ---" % (time.time() - start_time_cached))


+----+----------+
|view|avg_price |
+----+----------+
|7   |403005.77 |
|51  |788128.21 |
|15  |404673.3  |
|54  |798684.82 |
|11  |399548.12 |
|29  |397771.65 |
|69  |750537.94 |
|42  |396964.5  |
|73  |752861.18 |
|87  |1072285.2 |
|64  |767036.67 |
|3   |398867.6  |
|30  |397862.0  |
|34  |401419.75 |
|59  |791453.0  |
|8   |398592.71 |
|28  |402124.62 |
|22  |402022.68 |
|85  |1056336.74|
|35  |401934.21 |
+----+----------+
only showing top 20 rows

--- Parquet Runtime: 1.9303221702575684 seconds ---
+----+----------+
|view|avg_price |
+----+----------+
|7   |403005.77 |
|51  |788128.21 |
|15  |404673.3  |
|54  |798684.82 |
|11  |399548.12 |
|29  |397771.65 |
|69  |750537.94 |
|42  |396964.5  |
|87  |1072285.2 |
|73  |752861.18 |
|64  |767036.67 |
|3   |398867.6  |
|30  |397862.0  |
|34  |401419.75 |
|59  |791453.0  |
|8   |398592.71 |
|28  |402124.62 |
|22  |402022.68 |
|85  |1056336.74|
|16  |399586.53 |
+----+----------+
only showing top 20 rows

--- Cached Runtime: 1.0768680572

In [27]:
# 14. Uncache the home_sales temporary table.
spark.sql("UNCACHE TABLE home_sales_view")


DataFrame[]

In [28]:
# 15. Check if the home_sales is no longer cached

spark.catalog.isCached('home_sales_view')

False